In [1]:
#prelims
import polars as pl
import pandas as pd
import geopandas as gpd
import time
import plotly_express as px
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import glob

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=200);

# Port Performance Index Project - Main Notebook

This notebook presents the primary data analysis related to the PPI Project; see the README and other files in the [repo](https://github.com/epistemetrica/Port-Performance-Project) for full details. 

## Data Processing

AIS data is ingested from the Marine Cadastre database via the scripts found in the vessel data folder. Here, we combine the AIS data with port-level data and prepare for analysis. 

### AIS Data

The vessel locations and status (e.g., "under way", "anchored", "moored") data include all AIS messages; for the purposes of the PPI, we only need to know when a vessel *changes* status, so all other observations are dropped. 

We also limit the data for the PPI to vessels over 100m in length. 

In [2]:
#init list of lazyframes
lfs = []
#process each parquet file individually into lazyframes
for file in glob.glob('ais data/data/ais_clean/*.parquet'):
    try:
        #check file integrity 
        pl.scan_parquet(file).collect_schema()
        #read file
        lf = (
            pl.scan_parquet(file)
            #drop smaller vessels
            .filter(pl.col('length')>100)
            #sort by vessel and time
            .sort(['mmsi', 'time'])
            #indicate whether status is the same as previous row (Fill value needed to avoid status 0 evaluating as equal to false)
            .with_columns(
                status_change = (
                    pl.col('status').ne(pl.col('status').shift(fill_value=20))
                    .over('mmsi')
                ),
                status_previous = pl.col('status').shift().over('mmsi')
            )
            #keep only new status pings
            .filter(pl.col('status_change')==True)
            #drop change col
            .drop('status_change')
            #create duration column
            .with_columns(
                status_duration = (pl.col('time').shift(-1) - pl.col('time'))
                .over('mmsi').dt.total_minutes()
            )
        )
        #append to list of lazyframes
        lfs.append(lf)
    except:
        print(f'{file} failed')

#collect all lazyframes
dfs = pl.collect_all(lfs)

#create single pandas dataframe
ais_gdf = (
    #concat dfs
    pl.concat(dfs, how='diagonal_relaxed')
    #convert to pandas
    .to_pandas()
)

#convert to geopandas dataframe
ais_gdf = (
    #convert to geodataframe
    gpd.GeoDataFrame(
        ais_gdf,
        geometry=gpd.points_from_xy(ais_gdf.lon, ais_gdf.lat, crs='EPSG:4326')
    )
    #convert to WGS84 pseudo-mercator
    .to_crs(3857)
    #drop old lat lon cols
    .drop(['lat', 'lon'], axis=1)
)

ais data/data/ais_clean/ais_2024_09.parquet failed
ais data/data/ais_clean/ais_2024_10.parquet failed
ais data/data/ais_clean/ais_2024_11.parquet failed
ais data/data/ais_clean/ais_2024_08.parquet failed
ais data/data/ais_clean/ais_2024_12.parquet failed
ais data/data/ais_clean/ais_2024_07.parquet failed
ais data/data/ais_clean/ais_2024_06.parquet failed
ais data/data/ais_clean/ais_2024_04.parquet failed
ais data/data/ais_clean/ais_2024_05.parquet failed


### Port and Dock Data

Locations and descriptions for each dock and port come from the BTS and USACE online databases. 

In [3]:
#load port data
ports_gdf = (
    #read in shape file downloaded from BTS
    gpd.read_file('port data/Principal_Ports/Principal_Ports.shp')
    #drop unneeded columns
    .drop([
        'FID', #randomly assigned table id
        'PORT', #unknown numeric ID - not CBP or UN code
        'FOREIGN_','EXPORTS', 'IMPORTS', 'DOMESTIC' #breadown of total vol (tons)
    ], axis=1)
)
#set col names to pythonic lowercase
ports_gdf.columns = ports_gdf.columns.str.lower()

#load dock data
docks_gdf = (
    #read in shape file downloaded from USACE
    gpd.read_file('port data/Dock/Dock.shp')
    #drop unneeded columns
    .drop([
        'FID', #randomly assigned table id
        'LONGITUDE', 'LATITUDE', #already coded in 'geometry' 
        'LOCATION_D', #text description of dock location
        'STREET_ADD','ZIPCODE', #street address details
        'PSA_NAME', #statistical area name, rarely used
        'COUNTY_NAM', 'COUNTY_FIP', 'CONGRESS', 'CONGRESS_F', #county and congress info
        'MILE', 'BANK', 'LATITUDE1', 'LONGITUDE1', #redundant locaation data
        'OPERATORS', 'OWNERS', #owner info
        'PURPOSE', #long-form text description of dock uses
        'DOCK', #unknown number (not unique to each row/dock)
        'HIGHWAY_NO', 'RAILWAY_NO', 'LOCATION', #redundant location info
        'COMMODITIE', 'CONSTRUCTI','MECHANICAL', 'REMARKS', 'VERTICAL_D', 
        'DEPTH_MIN', 'DEPTH_MAX','BERTHING_L', 'BERTHING_T', 'DECK_HEIGH', 
        'DECK_HEI_1', #these are rarely used stats on construction
        'SERVICE_IN','SERVICE_TE', #rarely used indicators of data entry date 
    ], axis=1)
    #rename cols for clarity
    .rename(columns={
        'NAV_UNIT_I':'nav_unit_id',
        'NAV_UNIT_N':'nav_unit_name',
        'FACILITY_T':'facility_type',
        'CITY_OR_TO':'city',
        'STATE_POST':'state'
    })
)
#set col names to pythonic lowercase
docks_gdf.columns = docks_gdf.columns.str.lower()

In [4]:
docks_gdf.head()

,nav_unit_id,unlocode,nav_unit_name,facility_type,city,state,wtwy_name,port_name,geometry
0,01Y8,None,"YUTANA BARGE LINES, ST. MICHAELS DK",Dock,SAINT MICHAEL,AK,"Norton Sound, AK",None,POINT (-18036849.964 9218097.559)
1,01Y9,WMO,STEBBINS VILLAGE,Dock,WHITE MOUNTAIN,AK,"Norton Sound, AK",None,POINT (-18065988.397 9228900.858)
2,01YB,None,CANDIANA LIGHT WASH,None,CORBETT,WA,"Columbia River between Vancouver, WA and The D...","Port of Portland, OR",POINT (-13602488.476 5710656.422)
3,01YC,None,CAPE HORN WASH,None,CORBETT,WA,"Columbia River between Vancouver, WA and The D...",None,POINT (-13602673.934 5711970.977)
4,01YD,None,PHOCA ROCK,None,CORBETT,OR,"Columbia River between Vancouver, WA and The D...","Port of Portland, OR",POINT (-13601180.027 5712054.147)


### Matching

We first match each of the anchored and moored points from the AIS data with the nearest Port. 

In [5]:
start = time.time()
stops_gdf = (
    #filter to only moorings
    ais_gdf[ais_gdf.status == 5]
    #join in nearest port to each ais message
    .sjoin_nearest(ports_gdf, how='left', exclusive=True)
    #drop unneeded cols
    .drop(['index_right', 'total'], axis=1)
    #rename cols for clarity
    .rename({'rank':'port_rank', 'type':'port_type'}, axis=1)
)
print(f'Spatial Join time on {len(stops_gdf)} AIS messages took {time.time()-start} seconds')

#create main df
main_gdf = (
    #merge stops back into AIS data
    ais_gdf.merge(stops_gdf, how='left')
    #sort by vessel then time of message
    .sort_values(by=['mmsi', 'time'])
)
#backfill port info except geometry (normal pandas syntax not supported for gpd geometry)
main_gdf[['port_type','port_name','port_rank']] = (
    main_gdf[['port_type','port_name','port_rank']].bfill()
)
#merge port geometries into main (NOTE backfill not supported for gpd geometry, hence the separate merge step)
main_gdf = main_gdf.merge(ports_gdf[['port_name', 'geometry']], 
                          on='port_name', how='left', suffixes=[None, '_port'])
#compute distance from message loc to port loc
main_gdf['port_dist'] = main_gdf['geometry'].distance(main_gdf['geometry_port'])

Spatial Join time on 651437 AIS messages took 2.207688331604004 seconds


Next, we match similary to the nearest dock

In [6]:
start = time.time()
dockstops_gdf = (
    #filter to only moorings
    ais_gdf[ais_gdf.status == 5]
    #join in nearest port to each ais message
    .sjoin_nearest(
        #keep only dock id
        docks_gdf[['nav_unit_id', 'geometry']], 
        how='left')
    #drop unneeded cols
    .drop(['index_right'], axis=1)
)
print(f'Spatial Join time on {len(dockstops_gdf)} AIS messages took {time.time()-start} seconds')

#create main df
main_docks_gdf = (
    #merge stops back into AIS data
    ais_gdf.merge(dockstops_gdf, how='left')
    #sort by vessel then time of message
    .sort_values(by=['mmsi', 'time'])
)
#backfill dock info except geometry (normal pandas syntax not supported for gpd geometry)
main_docks_gdf[['nav_unit_id']] = (
    main_docks_gdf[['nav_unit_id']].bfill()
)
#merge dock geometries into main (NOTE backfill not supported for gpd geometry, hence the separate merge step)
main_docks_gdf = main_docks_gdf.merge(docks_gdf[['nav_unit_id', 'geometry']], 
                          on='nav_unit_id', how='left', suffixes=[None, '_dock'])
#compute distance from message loc to dock loc
main_docks_gdf['dock_dist'] = main_docks_gdf['geometry'].distance(main_docks_gdf['geometry_dock'])

Spatial Join time on 685969 AIS messages took 2.9290947914123535 seconds


In [7]:
#create year and month cols for convenience
main_docks_gdf['year'] = main_docks_gdf['time'].dt.year
main_docks_gdf['month'] = main_docks_gdf['time'].dt.strftime('%Y%m')

In [8]:
#create year and month cols for convenience
main_gdf['year'] = main_gdf['time'].dt.year
main_gdf['month'] = main_gdf['time'].dt.strftime('%Y%m')

In [9]:
#inspect
display(main_gdf.head())
main_gdf.info()

,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,status_previous,status_duration,geometry,port_type,port_name,port_rank,geometry_port,port_dist,year,month
0,-3.674380e+08,2023-05-27 00:06:10,13.3,165.5,167.0,0.0,MAUNAWILI,70.0,9268538.0,217.0,32.0,12.5,70.0,NaN,NaN,POINT (16076544.901 1532352.227),C,"Port of Palm Beach District, FL",116.0,POINT (-8911422.461 3094645.864),2.503676e+07,2023,202305
1,3.576870e-01,2023-04-14 00:06:22,12.8,171.9,167.0,0.0,PACIFIC CONDOR,70.0,9216339.0,117.0,20.0,7.5,70.0,NaN,NaN,POINT (16035316.614 1367477.099),C,"Port of Palm Beach District, FL",116.0,POINT (-8911422.461 3094645.864),2.500646e+07,2023,202304
2,3.000000e+00,2017-12-10 18:10:49,10.3,240.0,242.0,0.0,MAERSK PRIVILEGE,80.0,3.0,240.0,42.0,NaN,NaN,NaN,9136.0,POINT (-8374535.236 3066951.160),C,"Port of Palm Beach District, FL",116.0,POINT (-8911422.461 3094645.864),5.376010e+05,2017,201712
3,3.000000e+00,2017-12-17 02:27:29,0.0,60.0,35.0,1.0,MAERSK PRIVILEGE,80.0,3.0,240.0,42.0,-12.6,80.0,0.0,66.0,POINT (-10021529.366 3474701.894),C,"Port of Palm Beach District, FL",116.0,POINT (-8911422.461 3094645.864),1.173363e+06,2017,201712
4,3.000000e+00,2017-12-17 03:34:09,1.0,46.0,41.0,0.0,MAERSK PRIVILEGE,80.0,3.0,240.0,42.0,-12.6,80.0,1.0,NaN,POINT (-10021509.329 3474712.155),C,"Port of Palm Beach District, FL",116.0,POINT (-8911422.461 3094645.864),1.173347e+06,2017,201712


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2403395 entries, 0 to 2403394
Data columns (total 23 columns):
 #   Column           Dtype         
---  ------           -----         
 0   mmsi             float64       
 1   time             datetime64[us]
 2   speed            float64       
 3   course           float64       
 4   heading          float64       
 5   status           float64       
 6   vessel_name      category      
 7   vessel_type      float64       
 8   imo              float64       
 9   length           float64       
 10  width            float64       
 11  draft            float64       
 12  cargo            float64       
 13  status_previous  float64       
 14  status_duration  float64       
 15  geometry         geometry      
 16  port_type        object        
 17  port_name        object        
 18  port_rank        float64       
 19  geometry_port    geometry      
 20  port_dist        float64       
 21  year             int32 

## Exploratory Analysis

### Vessel calls in previous 12 months

In [10]:
#get most recent twelve month window that appears in the data
latest_date = main_gdf.time.max()
year_before = latest_date - pd.DateOffset(months=12)

#get port lat and long to preserve geometries in polars df
main_gdf['lat'] = main_gdf.set_geometry('geometry_port').to_crs("EPSG:4326").geometry_port.y
main_gdf['lon'] = main_gdf.set_geometry('geometry_port').to_crs("EPSG:4326").geometry_port.x

annual_gdf = (
    #convert main gdf to polars
    pl.DataFrame(
        main_gdf.drop(['geometry', 'geometry_port'], axis=1))
    #filter to most recent 12 months
    .filter(pl.col('time')>= year_before)
    .with_columns(
        #get monthly avg vessels
        vessels_avg = pl.col('mmsi').n_unique().over('port_name', 'month'),
        status_duration = pl.col('status_duration')
    )
    #aggregate
    .group_by('port_name')
    .agg(
        #keep lat and long
        lat = pl.col('lat').first(),
        lon = pl.col('lon').first(),
        #get avg vessels per month
        vessels_avg = pl.col('vessels_avg').mean(),
        #get median time at berth in hours
        time_at_berth_avg = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60,
        #get median time at anchor in hours
        time_at_anchor_avg = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60
    )
    #convert to pandas to that geopandas is happy
    .to_pandas()
)

#convert to geodataframe
annual_gdf = (
    gpd.GeoDataFrame(
        annual_gdf, geometry=gpd.points_from_xy(annual_gdf.lon, annual_gdf.lat),
        crs=3857
    )
)

### Monthly Port Statistics

In [ ]:
main_gdf[main_gdf.status==5].groupby(['port_name','month'],observed=True)[['status_duration']].median(numeric_only=True)

In [12]:
px.line(main_gdf[main_gdf.status==5].groupby(['port_name','year'],observed=True)['status_duration'].median(numeric_only=True).reset_index().sort_values('year'),
        x='year',y='status_duration',color='port_name', labels={'status_duration':'Median time at berth'}
)

In [13]:
px.line(main_gdf[main_gdf.status==5].groupby(['port_name','month'],observed=True)['status_duration'].median(numeric_only=True).reset_index().sort_values('month'),
        x='month',y='status_duration',color='port_name', labels={'status_duration':'Median time at berth'}
)

In [14]:
monthly_df = (
    #convert to polars
    pl.DataFrame(main_gdf.drop(['geometry', 'geometry_port'], axis=1))
    #agg over ports and months
    .group_by('port_name', 'month')
    .agg(
        #keep lat and long
        lat = pl.col('lat').first(),
        lon = pl.col('lon').first(),
        #get monthly avg vessels
        vessels_avg = pl.col('mmsi').n_unique(),
        #get average time at berth
        time_at_berth_avg = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60,
        #get average time at anchor
        time_at_anchor_avg = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60
    )
)

#get top 5 ports
top5ports = pl.Series(ports_gdf.sort_values('rank').head().port_name)

### Visualizations

In [15]:
#scatterplot
fig = px.scatter_geo(
    annual_gdf,
    lon='lon',
    lat='lat',
    size='vessels_avg',
    color='time_at_berth_avg',
    range_color=[0,50],
    hover_name='port_name',
    size_max=30,
    title='Avg Vessels per month & Median Hours at Berth (previous 12 months)',
    color_continuous_scale=px.colors.sequential.Viridis,
    width=1000,
    height=600,
    labels={
        'time_at_berth_avg':'Hours at Berth'
    }
)

# Fit the view to ports
fig.update_geos(fitbounds="locations")

# Add footnote using add_annotation
fig.add_annotation(
    text="Note: Circle size corresponds to averages vessels per month",  # Footnote text
    xref="paper", yref="paper",  # Position relative to the plot area
    x=0, y=0-0.05,  # Adjust to footnote position
    showarrow=False,  # No arrow, just text
    font=dict(size=14, color="black"),  # Customize the font style
    align="left"
)

# Show the figure
fig.show()

In [16]:
#scatterplot
fig = px.scatter_geo(
    annual_gdf,
    lon='lon',
    lat='lat',
    size='vessels_avg',
    color='time_at_anchor_avg',
    range_color=[0,50],
    hover_name='port_name',
    size_max=30,
    title='Avg Vessels per month & Median Hours at Anchor (previous 12 months)',
    color_continuous_scale=px.colors.sequential.Viridis,
    width=1000,
    height=600,
    labels={
        'time_at_anchor_avg':'Hours at Anchor'
    }
)

# Fit the view to ports
fig.update_geos(fitbounds="locations")

# Add footnote using add_annotation
fig.add_annotation(
    text="Note: Circle size corresponds to averages vessels per month",  # Footnote text
    xref="paper", yref="paper",  # Position relative to the plot area
    x=0, y=0-0.05,  # Adjust to footnote position
    showarrow=False,  # No arrow, just text
    font=dict(size=14, color="black"),  # Customize the font style
    align="left"
)

# Show the figure
fig.show()

In [17]:
px.line(
    monthly_df
    #restrict to top 5 ports
    .filter(pl.col('port_name').is_in(top5ports))
    #month in dt format
    .with_columns(pl.col('month').str.strptime(pl.Date, format='%Y%m'))
    .sort(by='month'), 
    #plot specs
    x='month', y='vessels_avg', color='port_name',
    title='Average Vessels per month at Principal Ports',
    width=1000,
    height=500 
)

In [18]:
px.line(
    monthly_df
    #restrict to top 5 ports
    .filter(pl.col('port_name').is_in(top5ports))
    #month in dt format
    .with_columns(pl.col('month').str.strptime(pl.Date, format='%Y%m'))
    .sort(by='month'), 
    #plot specs
    x='month', y='time_at_anchor_avg', color='port_name',
    title='Median Time at Anchor at Principal Ports',
    width=1000,
    height=500 
)

In [19]:
px.line(
    #data with month in dt format
    monthly_df
    #restrict to top 5 ports
    .filter(pl.col('port_name').is_in(top5ports))
    #month in dt format
    .with_columns(pl.col('month').str.strptime(pl.Date, format='%Y%m'))
    .sort(by='month'), 
    #plot specs
    x='month', y='time_at_berth_avg', color='port_name',
    title='Median Time at Berth at Principal Ports',
    width=1000,
    height=500 
)

In [21]:
monthly_df.head()

port_name,month,lat,lon,vessels_avg,time_at_berth_avg,time_at_anchor_avg
str,str,f64,f64,u32,f64,f64
"""Manatee County Port, FL""","""201808""",27.633681,-82.564143,24,22.391667,21.025
"""Nikiski, AK""","""201907""",60.74793,-151.3144,11,6.591667,38.1
"""Richmond, CA""","""201603""",37.924237,-122.374233,3,18.525,null
"""Port of Los Angeles, CA""","""201501""",33.726635,-118.268099,18,111.241667,12.716667
"""San Juan, PR""","""202202""",18.432783,-66.096678,55,12.725,8.883333


In [31]:
px.bar(
    monthly_df
    .with_columns(year = pl.col('month').str.to_date('%Y%m').dt.year().cast(pl.Utf8))
    .group_by('year').agg(pl.col('vessels_avg').sum())
    .sort('year'),
    y='vessels_avg', x='year'
)

In [32]:
px.bar(
    monthly_df
    .with_columns(year = pl.col('month').str.to_date('%Y%m'))
    .group_by('month').agg(pl.col('vessels_avg').sum())
    .sort('month'),
    y='vessels_avg', x='month'
)